<a href="https://colab.research.google.com/github/amankiitg/Foundation_AI/blob/main/Beginners_Guide_to_Text_Generation_using_LSTMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
aashita_nyt_comments_path = kagglehub.dataset_download('aashita/nyt-comments')

print('Data source import complete.')


100%|██████████| 480M/480M [00:06<00:00, 78.1MB/s]

Extracting files...


Data source import complete.


# Beginners Guide to Text Generation using LSTMs

Text Generation is a type of Language Modelling problem. Language Modelling is the core problem for a number of of natural language processing tasks such as speech to text, conversational system, and text summarization. A trained language model learns the likelihood of occurrence of a word based on the previous sequence of words used in the text. Language models can be operated at character level, n-gram level, sentence level or even paragraph level. In this notebook, I will explain how to create a language model for generating natural language text by implement and training state-of-the-art Recurrent Neural Network.

### Generating News headlines

In this kernel, I will be using the dataset of [New York Times Comments and Headlines](https://www.kaggle.com/aashita/nyt-comments) to train a text generation language model which can be used to generate News Headlines


## 1. Import the libraries

As the first step, we need to import the required libraries:

In [2]:
!pip install tensorflow keras

from numpy.random import seed

# keras module for building LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
import tensorflow.keras.utils as ku

# set seeds for reproducability
from tensorflow.random import set_seed
set_seed(2)
seed(1)

import pandas as pd
import numpy as np
import string, os

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

## 2. Load the dataset

Load the dataset of news headlines

In [3]:
import os

curr_dir = "/root/.cache/kagglehub/datasets/aashita/nyt-comments/"

# List all files
files = os.listdir(curr_dir)
print("Files in dataset directory:", files)

Files in dataset directory: ['versions', '13.complete']


In [4]:
import os

nested_dir = "/root/.cache/kagglehub/datasets/aashita/nyt-comments/versions/13/"
print("Files in nested directory:", os.listdir(nested_dir))


Files in nested directory: ['ArticlesJan2017.csv', 'CommentsApril2017.csv', 'CommentsFeb2017.csv', 'CommentsMarch2018.csv', 'ArticlesApril2018.csv', 'ArticlesFeb2017.csv', 'CommentsApril2018.csv', 'CommentsMarch2017.csv', 'CommentsJan2018.csv', 'CommentsFeb2018.csv', 'ArticlesApril2017.csv', 'ArticlesMarch2018.csv', 'ArticlesMay2017.csv', 'CommentsJan2017.csv', 'ArticlesFeb2018.csv', 'ArticlesMarch2017.csv', 'ArticlesJan2018.csv', 'CommentsMay2017.csv']


In [5]:
import os
import pandas as pd

curr_dir = "/root/.cache/kagglehub/datasets/aashita/nyt-comments/versions/13/"

all_headlines = []
processed_file = None  # To store the name of the file being processed

for filename in os.listdir(curr_dir):
    if "Articles" in filename:  # Adjust if necessary
        processed_file = filename  # Store the filename
        article_df = pd.read_csv(os.path.join(curr_dir, filename))
        all_headlines.extend(list(article_df.headline.values))
        break  # Stop after processing the first file

# Remove "Unknown" values
all_headlines = [h for h in all_headlines if h != "Unknown"]

# Print results
if processed_file:
    print(f"Extracted headlines from: {processed_file}")
else:
    print("No matching file found.")

print(f"Total headlines: {len(all_headlines)}")


Extracted headlines from: ArticlesJan2017.csv
Total headlines: 777


In [6]:
print("First 10 headlines:", all_headlines[:10])


First 10 headlines: [' G.O.P. Leadership Poised to Topple Obama’s Pillars', 'Fractured World Tested the Hope of a Young President', 'Little Troublemakers', 'Angela Merkel, Russia’s Next Target', 'Boots for a Stranger on a Bus', 'Molder of Navajo Youth, Where a Game Is Sacred', '‘The Affair’ Season 3, Episode 6: Noah Goes Home', 'Sprint and Mr. Trump’s Fictional Jobs', 'America  Becomes a Stan', 'Fighting Diabetes, and Leading by Example']


In [ ]:
#import shutil

#nested_dir = "/root/.cache/kagglehub/datasets/aashita/nyt-comments/versions/13/"
#shutil.make_archive("/root/nyt-comments", 'zip', nested_dir)

#from google.colab import files
#files.download("/root/nyt-comments.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 3. Dataset preparation

### 3.1 Dataset cleaning

In dataset preparation step, we will first perform text cleaning of the data which includes removal of punctuations and lower casing all the words.

In [7]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt

corpus = [clean_text(x) for x in all_headlines]
corpus[:10]

[' gop leadership poised to topple obamas pillars',
 'fractured world tested the hope of a young president',
 'little troublemakers',
 'angela merkel russias next target',
 'boots for a stranger on a bus',
 'molder of navajo youth where a game is sacred',
 'the affair season 3 episode 6 noah goes home',
 'sprint and mr trumps fictional jobs',
 'america  becomes a stan',
 'fighting diabetes and leading by example']

### 3.2 Generating Sequence of N-gram Tokens

Language modelling requires a sequence input data, as given a sequence (of words/tokens) the aim is the predict next word/token.  

The next step is Tokenization. Tokenization is a process of extracting tokens (terms / words) from a corpus. Python’s library Keras has inbuilt model for tokenization which can be used to obtain the tokens and their index in the corpus. After this step, every text document in the dataset is converted into sequence of tokens.


In [8]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    ## convert data to sequence of tokens
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences[:15]

[[73, 313],
 [73, 313, 616],
 [73, 313, 616, 3],
 [73, 313, 616, 3, 617],
 [73, 313, 616, 3, 617, 205],
 [73, 313, 616, 3, 617, 205, 314],
 [618, 38],
 [618, 38, 619],
 [618, 38, 619, 1],
 [618, 38, 619, 1, 206],
 [618, 38, 619, 1, 206, 4],
 [618, 38, 619, 1, 206, 4, 2],
 [618, 38, 619, 1, 206, 4, 2, 207],
 [618, 38, 619, 1, 206, 4, 2, 207, 50],
 [74, 620]]

In the above output [30, 507], [30, 507, 11], [30, 507, 11, 1] and so on represents the ngram phrases generated from the input data. where every integer corresponds to the index of a particular word in the complete vocabulary of words present in the text. For example

**Headline:** i stand  with the shedevils  
**Ngrams:** | **Sequence of Tokens**

<table>
<tr><td>Ngram </td><td> Sequence of Tokens</td></tr>
<tr> <td>i stand </td><td> [30, 507] </td></tr>
<tr> <td>i stand with </td><td> [30, 507, 11] </td></tr>
<tr> <td>i stand with the </td><td> [30, 507, 11, 1] </td></tr>
<tr> <td>i stand with the shedevils </td><td> [30, 507, 11, 1, 975] </td></tr>
</table>



### 3.3 Padding the Sequences and obtain Variables : Predictors and Target

Now that we have generated a data-set which contains sequence of tokens, it is possible that different sequences have different lengths. Before starting training the model, we need to pad the sequences and make their lengths equal. We can use pad_sequence function of Keras for this purpose. To input this data into a learning model, we need to create predictors and label. We will create N-grams sequence as predictors and the next word of the N-gram as label. For example:


Headline:  they are learning data science

<table>
<tr><td>PREDICTORS </td> <td>           LABEL </td></tr>
<tr><td>they                   </td> <td>  are</td></tr>
<tr><td>they are               </td> <td>  learning</td></tr>
<tr><td>they are learning      </td> <td>  data</td></tr>
<tr><td>they are learning data </td> <td>  science</td></tr>
</table>

In [9]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

Perfect, now we can obtain the input vector X and the label vector Y which can be used for the training purposes. Recent experiments have shown that recurrent neural networks have shown a good performance in sequence to sequence learning and text data applications. Lets look at them in brief.

## 4. LSTMs for Text Generation

![](http://www.shivambansal.com/blog/text-lstm/2.png)

Unlike Feed-forward neural networks in which activation outputs are propagated only in one direction, the activation outputs from neurons propagate in both directions (from inputs to outputs and from outputs to inputs) in Recurrent Neural Networks. This creates loops in the neural network architecture which acts as a ‘memory state’ of the neurons. This state allows the neurons an ability to remember what have been learned so far.

The memory state in RNNs gives an advantage over traditional neural networks but a problem called Vanishing Gradient is associated with them. In this problem, while learning with a large number of layers, it becomes really hard for the network to learn and tune the parameters of the earlier layers. To address this problem, A new type of RNNs called LSTMs (Long Short Term Memory) Models have been developed.

LSTMs have an additional state called ‘cell state’ through which the network makes adjustments in the information flow. The advantage of this state is that the model can remember or forget the leanings more selectively. To learn more about LSTMs, here is a great post. Lets architecture a LSTM model in our code. I have added total three layers in the model.

1. Input Layer : Takes the sequence of words as input
2. LSTM Layer : Computes the output using LSTM units. I have added 100 units in the layer, but this number can be fine tuned later.
3. Dropout Layer : A regularisation layer which randomly turns-off the activations of some neurons in the LSTM layer. It helps in preventing over fitting. (Optional Layer)
4. Output Layer : Computes the probability of the best possible next word as output

We will run this model for total 100 epoochs but it can be experimented further.

In [10]:
print(total_words)
print(max_sequence_len)

2217
21


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()

    # Add Input Embedding Layer
    model.add(Embedding(input_dim=total_words, output_dim=10, input_length=input_len))

    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))

    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

# Create and build the model
model = create_model(max_sequence_len, total_words)

# 🔥 Explicitly build the model by calling it on an example input
model.build(input_shape=(None, max_sequence_len - 1))

# Print model summary
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 20, 10)              │          22,170 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 100)                 │          44,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 2217)                │         223,917 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 290,487 (1.11 MB)

 Trainable params: 290,487 (1.11 MB)

 Non-trainable params: 0 (0.00 B)

Lets train our model now

In [12]:
model.fit(predictors, label, epochs=100, verbose=5)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

## 5. Generating the text

Great, our model architecture is now ready and we can train it using our data. Next lets write the function to predict the next word based on the input words (or seed text). We will first tokenize the seed text, pad the sequences and pass into the trained model to get predicted word. The multiple predicted words can be appended together to get predicted sequence.


In [13]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)

        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

## 6. Some Results

In [17]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_text(seed_text, next_words, model, max_sequence_len, tokenizer):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

        # 🔥 Use `predict()` and `argmax()` instead of `predict_classes()`
        predicted_probs = model.predict(token_list, verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)[0]  # Get the word index

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break

        seed_text += " " + output_word  # Append predicted word
    return seed_text.title()

# Example calls (ensure `tokenizer` is defined)
print(generate_text("united states", 5, model, max_sequence_len, tokenizer))
print(generate_text("president trump", 4, model, max_sequence_len, tokenizer))
print(generate_text("donald trump", 4, model, max_sequence_len, tokenizer))
print(generate_text("india and china", 4, model, max_sequence_len, tokenizer))
print(generate_text("new york", 4, model, max_sequence_len, tokenizer))
print(generate_text("science and technology", 5, model, max_sequence_len, tokenizer))
print(generate_text("aman and jayada", 5, model, max_sequence_len, tokenizer))


United States Jan Farmer 2017 Ever Only
President Trump Obama Not The World
Donald Trump Obama Cant Not The
India And China Beginning A Game House
New York Today A Birds Eye
Science And Technology Obama Not A World Of
Aman And Jayada The Womens Prison To Keep


## Improvement Ideas

The results can be improved further with following points:
- Adding more data
- Fine Tuning the network architecture
- Fine Tuning the network parameters
